<h>Test of subarray resource allocation, configuration, deconfiguration and deallocation.</h>

First defines all elements in chart. Configuration uses one subarray, two subarray beams, 
four apertures in two station. 

Physical hardware uses old devices (e.g. APIU) and includes 4 antennas, one TPM and one subrack. This is a nonstandard configuration, does not work in reality. 

In the final cluster the devices should be the correct ones, i.e. PASD and SpsStation. MccsStation must be refactored to reflect this, but as we stop the configuration propagation in the MccsStation it is sufficient.

In [2]:
import tango
import time
import json
import numpy as np

from ska_control_model import (
    AdminMode,
    CommunicationStatus,
    HealthState,
    PowerState,
    ResultCode,
    TaskStatus,
    SimulationMode,
    TestMode,
)

# for time conversion
from datetime import datetime,timezone
RFC_FORMAT = "%Y-%m-%dT%H:%M:%S.%fZ"
# define devices
controller = tango.DeviceProxy('low-mccs/control/control')
sa1 = tango.DeviceProxy('low-mccs/subarray/01')
sa2 = tango.DeviceProxy('low-mccs/subarray/02')
sab1 = tango.DeviceProxy('low-mccs/subarraybeam/01')
sab2 = tango.DeviceProxy('low-mccs/subarraybeam/02')
st1 = tango.DeviceProxy('low-mccs/station/001')
st2 = tango.DeviceProxy('low-mccs/station/002')
# st3 = tango.DeviceProxy('low-mccs/station/003')
# st4 = tango.DeviceProxy('low-mccs/station/004')
sb1 = tango.DeviceProxy('low-mccs/beam/001')
sb2 = tango.DeviceProxy('low-mccs/beam/002')
sb3 = tango.DeviceProxy('low-mccs/beam/003')
sb4 = tango.DeviceProxy('low-mccs/beam/004')
# a1 = tango.DeviceProxy('low-mccs/antenna/000001')
# a2 = tango.DeviceProxy('low-mccs/antenna/000002')
# a3 = tango.DeviceProxy('low-mccs/antenna/000003')
# a4 = tango.DeviceProxy('low-mccs/antenna/000004')
# a5 = tango.DeviceProxy('low-mccs/antenna/000005')
# a6 = tango.DeviceProxy('low-mccs/antenna/000006')
# a7 = tango.DeviceProxy('low-mccs/antenna/000007')
# a8 = tango.DeviceProxy('low-mccs/antenna/000008')
# ap1 = tango.DeviceProxy('low-mccs/apiu/001')
spsst1 = tango.DeviceProxy('low-mccs/spsstation/ci-1')
spsst2 = tango.DeviceProxy('low-mccs/spsstation/ci-2')
sr1 = tango.DeviceProxy('low-mccs/subrack/ci-1-1')
sr2 = tango.DeviceProxy('low-mccs/subrack/ci-2-1')
t1 = tango.DeviceProxy('low-mccs/tile/ci-1-13')
t2 = tango.DeviceProxy('low-mccs/tile/ci-1-17')
t3 = tango.DeviceProxy('low-mccs/tile/ci-2-13')
t4 = tango.DeviceProxy('low-mccs/tile/ci-2-17')
devices = [spsst1, spsst2, sr1, sr2, t1, t2, t3, t4, st1, st2, sb1, sb2, sb3, sb4, sab1, sab2, sa1, sa2]
hwdevices = [spsst1, spsst2, sr1, sr2, t1, t2, t3, t4]

Put everything online and ON. Logging level for some devices is raised to DEBUG.  

In [2]:
for d in devices:
    d.adminmode = 0
    d.logginglevel = 5
for d in hwdevices:
    d.on() 
controller.adminmode=0
controller.logginglevel = 5
sa1.logginglevel = 5
sab1.logginglevel = 5
sb1.logginglevel = 5

Waiting for the system to come up

In [ ]:
timeout = 20   # seconds
while controller.state() != tango._tango.DevState.ON:
    time.sleep(1)
    timeout -= 1
    if timeout == 0: 
        break
if timeout == 0:
    print("Error: timeout in waiting for controller to coming ON")
for d in devices:
    print(f"{d.name()}: {d.status()} health {HealthState(d.healthState)}")
print(f"{controller.name()}: {controller.status()}")

if timeout > 0: 
    if any(state !='Synchronised') for state in station.tileprogrammingstate for station in [spsst1,spsst2]:
        start_time = datetime.strftime(datetime.fromtimestamp(int(time.time())+3), RFC_FORMAT)
        for station in [spsst1, spsst2]:
            station.StartAcquisition(json.dumps({"start_time": start_time}))
    #
    # check that synchronization worked
    #
    time.sleep(4)
for station in [spsst1, spsst2]:
    print(f'{station.name()}: state = {station.tileprogrammingstate}')

low-mccs/spsstation/ci-1: The device is in ON state. health 2
low-mccs/spsstation/ci-2: The device is in ON state. health 2
low-mccs/subrack/ci-1-1: The device is in ON state. health 2
low-mccs/subrack/ci-2-1: The device is in ON state. health 2
low-mccs/tile/ci-1-13: The device is in ON state. health 2
low-mccs/tile/ci-1-17: The device is in ON state. health 2
low-mccs/tile/ci-2-13: The device is in ON state. health 3
low-mccs/tile/ci-2-17: The device is in ON state. health 3
low-mccs/station/001: The device is in ON state. health 2
low-mccs/station/002: The device is in ON state. health 2
low-mccs/beam/001: The device is in ON state. health 2
low-mccs/beam/002: The device is in ON state. health 2
low-mccs/beam/003: The device is in ON state. health 2
low-mccs/beam/004: The device is in ON state. health 2
low-mccs/subarraybeam/01: The device is in ON state. health 2
low-mccs/subarraybeam/02: The device is in ON state. health 2
low-mccs/subarray/01: The device is in ON state. health 2


<title> Allocate and deallocate resources</title>

1) Assess that the controller has no assigned resources to subarray 1, neither has the subarray

2) Allocate resources to the subarray and check that these have been allocated in the controller, and assigned to the subarray.
    Subarray is composed of two subarray beams, 32 channels and 2 apertures each. 
    Station 1 hosts the 2 apertures of beam 1 and one of beam 2. Station 2 hosts the 2nd aperture of beam 2.

3) Release the resources and check they have been released. 

In [5]:
print(f"Controller resources allocated to subarray 1: {controller.GetAssignedResources(1)}")
print(f"Subarray resources\n- Stations:       {sa1.stationFQDNs}")
print(f"- Subarray beams: {sa1.subarraybeamfqdns}")
print(f"- Station beams:  {sa1.stationbeamfqdns}")
print(f"- All assigned resources: {json.loads((sa1.assignedresources))}")

Controller resources allocated to subarray 1: {}
Subarray resources
- Stations:       ()
- Subarray beams: ()
- Station beams:  ()
- All assigned resources: {'interface': 'https://schema.skao.int/ska-low-mccs-assignedresources/1.0', 'subarray_beam_ids': [], 'station_beam_ids': [], 'station_ids': [], 'apertures': [], 'channels': [0]}


In [4]:
args = {
    "subarray_id": 1, 
    "subarray_beams": [
        {
            "subarray_beam_id": 1, 
            "apertures": [
                {
                    "station_id": 1,
                    "aperture_id": "AP001.01"
                },
                {
                    "station_id": 1,
                    "aperture_id": "AP001.02"
                },
            ], 
            "number_of_channels": 32
        },
        {
            "subarray_beam_id": 2, 
            "apertures": [
                {
                    "station_id": 1,
                    "aperture_id": "AP001.03"
                },
                {
                    "station_id": 2,
                    "aperture_id": "AP002.04"
                },
            ], 
            "number_of_channels": 32
        },
    ]}
controller.Allocate(json.dumps(args))

[array([2], dtype=int32), ['1698683688.8465571_160872017842122_Allocate']]

In [5]:
print(f"Controller resources allocated to subarray 1: {controller.GetAssignedResources(1)}")
print(f"Subarray resources\n- Stations:       {sa1.stationFQDNs}")
print(f"- Subarray beams: {sa1.subarraybeamfqdns}")
print(f"- Station beams:  {sa1.stationbeamfqdns}")
print(f"- All assigned resources: {json.loads((sa1.assignedresources))}")

Controller resources allocated to subarray 1: {"subarray_beams": ["low-mccs/subarraybeam/01", "low-mccs/subarraybeam/02"], "station_beams": ["low-mccs/beam/001", "low-mccs/beam/002", "low-mccs/beam/003", "low-mccs/beam/004"]}
Subarray resources
- Stations:       ('low-mccs/station/001', 'low-mccs/station/002')
- Subarray beams: ('low-mccs/subarraybeam/01', 'low-mccs/subarraybeam/02')
- Station beams:  ('low-mccs/beam/001', 'low-mccs/beam/002', 'low-mccs/beam/003', 'low-mccs/beam/004')
- All assigned resources: {'interface': 'https://schema.skao.int/ska-low-mccs-assignedresources/1.0', 'subarray_beam_ids': ['1', '2'], 'station_beam_ids': ['1', '2', '3', '4'], 'station_ids': ['1', '2'], 'apertures': ['AP001.01', 'AP001.02', 'AP001.03', 'AP002.04'], 'channels': [64]}


In [8]:
controller.ReleaseAll()

[array([0], dtype=int32), ['Release command completed OK']]

In [25]:
print(f"Controller resources allocated to subarray 1: {controller.GetAssignedResources(1)}")
print(f"Subarray resources\n- Stations:       {sa1.stationFQDNs}")
print(f"- Subarray beams: {sa1.subarraybeamfqdns}")
print(f"- Station beams:  {sa1.stationbeamfqdns}")
print(f"- All assigned resources: {json.loads((sa1.assignedresources))}")

Controller resources allocated to subarray 1: {}
Subarray resources
- Stations:       ()
- Subarray beams: ()
- Station beams:  ()
- All assigned resources: {'interface': 'https://schema.skao.int/ska-low-mccs-assignedresources/1.0', 'subarray_beam_ids': [], 'station_beam_ids': [], 'station_ids': [], 'apertures': [], 'channels': [0]}


Reallocate the same resources to subarray 1. Configure subarray 1 with multiple bands and independent pointing in each beam. Substations cannot yet be defined (there is no aperture weight database). 

In [24]:
controller.Allocate(json.dumps(args))

[array([3], dtype=int32), ['Command failed']]

In [31]:
sa_configuration = {
    "subarray_beams": [
        {
            "subarray_beam_id": 1,
            "update_rate": 5.0,
            "logical_bands": [
                {
                    "start_channel": 66,
                    "number_of_channels": 16
                },
                {
                    "start_channel": 262,
                    "number_of_channels": 16
                }
            ],
            "sky_coordinates": {
                "reference_frame": "ICRS",
                "timestamp": "2023-04-05T12:34:56.000Z",
                "c1": 123.45,
                "c1_rate": 0.0,
                "c2": -12.34,
                "c2_rate": 0.0,
            },
            "apertures": [
                {"aperture_id": "AP001.01"},
                {"aperture_id": "AP001.02"},
            ]
        },
        {
            "subarray_beam_id": 2,
            "update_rate": 10.0,
            "logical_bands": [
                {
                    "start_channel": 166,
                    "number_of_channels": 16
                },
                {
                    "start_channel": 200,
                    "number_of_channels": 16
                }
            ],
            "sky_coordinates": {
                "reference_frame": "ICRS",
                "timestamp": "2023-04-05T12:34:56.000Z",
                "c1": 234.45,
                "c1_rate": 0.001,
                "c2": 23.34,
                "c2_rate": 0.001,
            },
            "apertures": [
                {"aperture_id": "AP001.03"},
                {"aperture_id": "AP002.04"},
            ]
        },
    ]
}
sa1.Configure(json.dumps(sa_configuration))

[array([2], dtype=int32), ['1698735533.6456742_122779193411418_Configure']]

Check that the configuration arrived to the station beams. Check that: 
1) Each Station beam has correct aperture, station and pointing
2) Each Station beam has built a table of beamformer channel blocks. Table entry specifies:
    1) table entry (station channel block)
    2) start physical channel. 0 means unused
    3) hardware beam used by station
    4) subarray ID
    5) start subarray logical channel
    6) subarray beam ID
    7) substation ID
    8) aperture ID
3) Each Station has assembled the beamformer table from all subarray beams

In [32]:
for sb in [sb1, sb2, sb3, sb4]:
    print(f"beam {sb.beamId}: subarray beam {sb.subarrayId}:{sb.logicalBeamId} ID {sb.apertureId} assigned to station {sb.stationId} ({sb.stationFqdn})")
    print(f"Pointing ({sb.pointingReferenceFrame}) {sb.desiredPointing} at time {sb.pointingTimestamp} - HW beam {sb.hardwareBeamId}")
    print(f"Channel table: \n{sb.channels}")
    print(f"Beam {sb.name()} is in state {sb.obsState}")
for st in [st1, st2]:
    print(f"{st.name()}: {st.numberOfChannels} total channels. Channel table:\n{st.beamformerTable}") 

beam 1: subarray beam 1:1 ID AP001.01 assigned to station 1 (low-mccs/station/001)
Pointing (ICRS) [123.45   0.   -12.34   0.  ] at time 2023-04-05T12:34:56.000Z - HW beam 0
Channel table: 
[[  0  66   0   1   0   1   1 101]
 [  1  74   0   1   8   1   1 101]
 [  2 262   0   1  16   1   1 101]
 [  3 270   0   1  24   1   1 101]]
Beam low-mccs/beam/001 is in state 4
beam 2: subarray beam 1:1 ID AP001.02 assigned to station 1 (low-mccs/station/001)
Pointing (ICRS) [123.45   0.   -12.34   0.  ] at time 2023-04-05T12:34:56.000Z - HW beam 1
Channel table: 
[[  4  66   1   1   0   1   2 102]
 [  5  74   1   1   8   1   2 102]
 [  6 262   1   1  16   1   2 102]
 [  7 270   1   1  24   1   2 102]]
Beam low-mccs/beam/002 is in state 4
beam 3: subarray beam 1:2 ID AP001.03 assigned to station 1 (low-mccs/station/001)
Pointing (ICRS) [2.3445e+02 1.0000e-03 2.3340e+01 1.0000e-03] at time 2023-04-05T12:34:56.000Z - HW beam 2
Channel table: 
[[  8 166   2   1  32   2   3 103]
 [  9 174   2   1  40  

1) Deconfigure and reconfigure the subarray
2) Reconfigure an aready configured subarray
3) Deconfigure and deallocate everything
4) Turn off a TPM. The controller goes in Unknown state (partly ON, partly Off)

In [53]:
sa1.End()

[array([2], dtype=int32), ['1683145934.132142_270989487535411_End']]

In [12]:
sa_configuration = {
    "subarray_beams": [
        {
            "subarray_beam_id": 1,
            "update_rate": 5.0,
            "logical_bands": [
                {
                    "start_channel": 266,
                    "number_of_channels": 16
                }
            ],
            "sky_coordinates": {
                "reference_frame": "ICRS",
                "timestamp": "2023-04-05T12:34:56.000Z",
                "c1": 123.45,
                "c1_rate": 0.0,
                "c2": -12.34,
                "c2_rate": 0.0,
            },
            "apertures": [
                {"aperture_id": "AP1.1"},
                {"aperture_id": "AP1.2"},
            ]
        },
        {
            "subarray_beam_id": 2,
            "update_rate": 10.0,
            "logical_bands": [
                {
                    "start_channel": 100,
                    "number_of_channels": 24
                }
            ],
            "sky_coordinates": {
                "reference_frame": "ICRS",
                "timestamp": "2023-04-05T12:34:56.000Z",
                "c1": 234.45,
                "c1_rate": 0.001,
                "c2": 23.34,
                "c2_rate": 0.001,
            },
            "apertures": [
                {"aperture_id": "AP1.3"},
                {"aperture_id": "AP2.4"},
            ]
        },
    ]
}
sa1.Configure(json.dumps(sa_configuration))

PyDs_PythonError: jsonschema.exceptions.ValidationError: 'AP1.1' does not match '^AP(?!0{3})\\d{3}\\.\\d{2}$'

Failed validating 'pattern' in schema['properties']['subarray_beams']['items']['properties']['apertures']['items']['properties']['aperture_id']:
    {'description': 'Aperture ID, of the form APXXX.YY, XXX=station '
                    'YY=substation',
     'pattern': '^AP(?!0{3})\\d{3}\\.\\d{2}$',
     'type': 'string'}

On instance['subarray_beams'][0]['apertures'][0]['aperture_id']:
    'AP1.1'

(For more detailed information type: tango_error)


In [41]:
for sb in [sb1, sb2, sb3, sb4]:
    print(f"beam {sb.beamId}: subarray beam {sb.subarrayId}:{sb.logicalBeamId} ID {sb.apertureId} assigned to station {sb.stationId} ({sb.stationFqdn})")
    print(f"Pointing ({sb.pointingReferenceFrame}) {sb.desiredPointing} at time {sb.pointingTimestamp} - HW beam {sb.hardwareBeamId}")
    print(f"Channel table: \n{sb.channels}")    
    print(f"Beam {sb.name()} is in state {sb.obsState}")
for st in [st1, st2]:
    print(f"{st.name()}: {st.numberOfChannels} total channels. Channel table:\n{st.beamformerTable}") 

beam 1: subarray beam 1:1 ID AP001.01 assigned to station 1 (low-mccs/station/001)
Pointing (ICRS) [123.45   0.   -12.34   0.  ] at time 2023-04-05T12:34:56.000Z - HW beam 0
Channel table: 
[[  0  66   0   1   0   1   1 101]
 [  1  74   0   1   8   1   1 101]
 [  2 262   0   1  16   1   1 101]
 [  3 270   0   1  24   1   1 101]]
Beam low-mccs/beam/001 is in state 4
beam 2: subarray beam 1:1 ID AP001.02 assigned to station 1 (low-mccs/station/001)
Pointing (ICRS) [123.45   0.   -12.34   0.  ] at time 2023-04-05T12:34:56.000Z - HW beam 1
Channel table: 
[[  4  66   1   1   0   1   2 102]
 [  5  74   1   1   8   1   2 102]
 [  6 262   1   1  16   1   2 102]
 [  7 270   1   1  24   1   2 102]]
Beam low-mccs/beam/002 is in state 4
beam 3: subarray beam 1:2 ID AP001.03 assigned to station 1 (low-mccs/station/001)
Pointing (ICRS) [2.3445e+02 1.0000e-03 2.3340e+01 1.0000e-03] at time 2023-04-05T12:34:56.000Z - HW beam 2
Channel table: 
[[  8 166   2   1  32   2   3 103]
 [  9 174   2   1  40  

In [50]:
scan_id = 1234
start_time = datetime.strftime(datetime.fromtimestamp(int(time.time())+2), RFC_FORMAT)
sa1.scan(json.dumps({'scan_id': scan_id, 'start_time': start_time}))

[array([2], dtype=int32), ['1698737836.743625_172993090082020_Scan']]

In [13]:
sa1.end()

[array([2], dtype=int32), ['1698684362.2930324_29537374896676_End']]

In [57]:
for sb in [sb1, sb2, sb3, sb4]:
    print(f"beam {sb.beamId}: subarray beam {sb.subarrayId}:{sb.logicalBeamId} ID {sb.apertureId} assigned to station {sb.stationId} ({sb.stationFqdn})")
    print(f"Pointing ({sb.pointingReferenceFrame}) {sb.desiredPointing} at time {sb.pointingTimestamp} - HW beam {sb.hardwareBeamId}")
    print(f"Channel table: \n{sb.channels}")    
    print(f"Beam {sb.name()} is in state {sb.obsState}")
for st in [st1, st2]:
    print(f"{st.name()}: {st.numberOfChannels} total channels. Channel table:\n{st.beamformerTable}") 

beam 1: subarray beam 1:1 ID AP1.1 assigned to station 1 (low-mccs/station/001)
Pointing (topocentric) [0. 0. 0. 0.] at time 2000-01-01T00:00:00.000Z - HW beam 0
Channel table: 
[[  0   0   0   0   0   1   1 101]
 [  1   0   0   0   8   1   1 101]
 [  2   0   0   0  16   1   1 101]
 [  3   0   0   0  24   1   1 101]]
Beam low-mccs/beam/001 is in state 2
beam 2: subarray beam 1:1 ID AP1.2 assigned to station 1 (low-mccs/station/001)
Pointing (topocentric) [0. 0. 0. 0.] at time 2000-01-01T00:00:00.000Z - HW beam 1
Channel table: 
[[  4   0   1   0   0   1   2 102]
 [  5   0   1   0   8   1   2 102]
 [  6   0   1   0  16   1   2 102]
 [  7   0   1   0  24   1   2 102]]
Beam low-mccs/beam/002 is in state 2
beam 3: subarray beam 1:2 ID AP1.3 assigned to station 1 (low-mccs/station/001)
Pointing (topocentric) [0. 0. 0. 0.] at time 2000-01-01T00:00:00.000Z - HW beam 2
Channel table: 
[[  8   0   2   0  32   2   3 103]
 [  9   0   2   0  40   2   3 103]
 [ 10   0   2   0  48   2   3 103]
 [ 11

In [58]:
controller.Release('{}')

[array([0], dtype=int32), ['Release command completed OK']]

In [59]:
sa1.assignedresources

'{"interface": "https://schema.skao.int/ska-low-mccs-assignedresources/1.0", "subarray_beam_ids": [], "station_beam_ids": [], "station_ids": [], "apertures": [], "channels": [0]}'

In [60]:
t1.off()
time.sleep(0.5)
controller.State()

tango._tango.DevState.UNKNOWN

In [9]:
sa1.assignedresources

'{"interface": "https://schema.skao.int/ska-low-mccs-assignedresources/1.0", "subarray_beam_ids": ["1", "2"], "station_beam_ids": ["1", "2", "3", "4"], "station_ids": ["1", "2"], "apertures": ["AP001.01", "AP001.02", "AP001.03", "AP002.04"], "channels": [64]}'

In [11]:
sc = tango.DeviceProxy('low-mccs/stationcalibrator/ci-1')

In [14]:
sc.status()

'The device is in UNKNOWN state.'

In [13]:
sc.adminmode=0

In [33]:
print(spsst1.beamformertable.reshape(-1,7))

[[ 66   0   1   0   1   1 101]
 [ 74   0   1   8   1   1 101]
 [262   0   1  16   1   1 101]
 [270   0   1  24   1   1 101]
 [ 66   1   1   0   1   2 102]
 [ 74   1   1   8   1   2 102]
 [262   1   1  16   1   2 102]
 [270   1   1  24   1   2 102]
 [166   2   1  32   2   3 103]
 [174   2   1  40   2   3 103]
 [200   2   1  48   2   3 103]
 [208   2   1  56   2   3 103]
 [  0   0   0   0   0   0   0]
 [  0   0   0   0   0   0   0]
 [  0   0   0   0   0   0   0]
 [  0   0   0   0   0   0   0]
 [  0   0   0   0   0   0   0]
 [  0   0   0   0   0   0   0]
 [  0   0   0   0   0   0   0]
 [  0   0   0   0   0   0   0]
 [  0   0   0   0   0   0   0]
 [  0   0   0   0   0   0   0]
 [  0   0   0   0   0   0   0]
 [  0   0   0   0   0   0   0]
 [  0   0   0   0   0   0   0]
 [  0   0   0   0   0   0   0]
 [  0   0   0   0   0   0   0]
 [  0   0   0   0   0   0   0]
 [  0   0   0   0   0   0   0]
 [  0   0   0   0   0   0   0]
 [  0   0   0   0   0   0   0]
 [  0   0   0   0   0   0   0]
 [  0   

In [40]:
t1.beamformertable.reshape(-1,7)

array([[ 66,   0,   1,   0,   1,   1, 101],
       [ 74,   0,   1,   8,   1,   1, 101],
       [262,   0,   1,  16,   1,   1, 101],
       [270,   0,   1,  24,   1,   1, 101],
       [ 66,   1,   1,   0,   1,   2, 101],
       [ 74,   1,   1,   8,   1,   2, 101],
       [262,   1,   1,  16,   1,   2, 101],
       [270,   1,   1,  24,   1,   2, 101],
       [166,   2,   1,  32,   2,   3, 101],
       [174,   2,   1,  40,   2,   3, 101],
       [200,   2,   1,  48,   2,   3, 101],
       [208,   2,   1,  56,   2,   3, 101]], dtype=int32)

In [10]:
sab1.stationbeamids

array([1, 2], dtype=int32)

In [23]:
sab3.releaseallresources()

NameError: name 'sab3' is not defined
(For more detailed information type: python_error)


In [17]:
python_error

---------------------------------------------------------------------------
AttributeError                            Traceback (most recent call last)
Cell In [16], line 1
----> 1 sab1.release

File /opt/conda/lib/python3.10/site-packages/tango/device_proxy.py:452, in __DeviceProxy__getattr(self, name)
    449     if name_l in self.__get_pipe_cache():
    450         return self.read_pipe(name)
--> 452     raise AttributeError(name) from cause
    453 finally:
    454     del cause

AttributeError: release


In [23]:
spsst1.initialise()

[array([2], dtype=int32), ['1698685122.630704_263311607334922_Initialise']]

In [26]:
t1.statictimedelays = [0]*32

In [24]:
t1.statictimedelays=list(range(32))

In [27]:
t1.statictimedelays

array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.])

In [28]:
t1.initialise()
t2.initialise()

[array([2], dtype=int32), ['1698735374.059577_133966971971195_Initialise']]

In [35]:
table = spsst1.beamformertable

In [39]:
spsst1.SetBeamformerTable(table)

[array([0], dtype=int32), ['SetBeamFormerTable command completed OK']]

In [46]:
spsst1.SetCspIngest(json.dumps({'destination_ip': '10.0.0.98'}))

[array([0], dtype=int32), ['SetCspIngest command completed OK']]

In [47]:
sa1.obsstate

<obsState.READY: 4>

In [51]:
t1.isbeamformerrunning

True

In [62]:
conf = json.loads(t1.get40gcoreconfiguration(json.dumps({'core_id':0})))

In [71]:
t2.get40gcoreconfiguration(json.dumps({'core_id':0}))

'{"core_id": 0, "arp_table_entry": 0, "source_mac": 107752307294362, "source_ip": "10.0.0.154", "source_port": 61648, "destination_ip": "10.0.0.98", "destination_port": 4660}'

In [69]:
conf["core_id"] = 0
conf["source_ip"] = '10.0.0.152'
conf["destination_ip"] = '10.0.0.154'
t1.configure40gcore(json.dumps(conf))
conf["core_id"] = 1
conf["source_ip"] = '10.0.0.152'
conf["destination_ip"] = '10.0.0.154'
t1.configure40gcore(json.dumps(conf))

[array([0], dtype=int32), ['Configure40GCore command completed OK']]

In [73]:
t2.isbeamformerrunning

True

In [75]:
t1.startbeamformer('{}')

[array([0], dtype=int32), ['StartBeamformer command completed OK']]

In [3]:
spsst1.on()

[array([5], dtype=int32), ['Device is already in ON state.']]

In [9]:
sr1.status()

'The device is in UNKNOWN state.'

In [5]:
spsst2.on()

[array([2], dtype=int32), ['1698759377.9843724_264759503298762_On']]

In [10]:
spsst1.on()

[array([2], dtype=int32), ['1698759664.6342075_114643505047742_Off']]

In [13]:
t2.off()

[array([2], dtype=int32), ['1698762968.6610122_160752091154532_Off']]